# Assignment 6: Chemical Enviroment - Molecular Docking and Drug Design

### Due:  M 4-28-25 at 9:29 am

### Name:

This assignment has been split into three parts. Each part has a general procedure for their portion of the assignment as well as a discussion section. Please be sure to address all parts of the discussion as your responses to this section are what determine your grade on this assignment.

## Part A - Creating an AI agent
 
#### Objective

The object of this part of the assignment is to teach you how to create your own AI agent through showing you an example of how to do so.

#### Procedure

First we need to load the conda environment from the last assignment.

```
module load anaconda
conda activate assignment5
pip install langchain
pip install langgraph
pip install langchain-openai

```

Agents are essestially chatbots that have access to tools. Tools are custom python functions that you define that agents determine what to put in and observe what is outputted. 
Here we will create a tool from the pubchem rest api. Here is a link to the tutorial of how the api is structured: https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest-tutorial.

In [ ]:
import getpass
import os

#Set API key if it's not already set
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = "put_key_here"

from langchain.chat_models import init_chat_model

In [2]:
from langchain.tools import tool
import requests

@tool
def get_compound_properties(compound_name, properties="MolecularFormula,MolecularWeight"):
    """
    Fetches specified properties of a compound from the PubChem REST API by name.

    Parameters:
        compound_name (str): The name of the compound (e.g., "aspirin").
        properties (str): Comma-separated properties to fetch (default: "MolecularFormula,MolecularWeight").

    Returns:
        dict: A dictionary containing the requested properties or an error message.
    """
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name"
    url = f"{base_url}/{compound_name}/property/{properties}/JSON"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()
        
        # Extract properties
        return data.get("PropertyTable", {}).get("Properties", [{}])[0]
    
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}
    
@tool
def get_bioassay_names(compound_name):
    """
    Fetches bioassay names associated with a given compound from the PubChem REST API.

    Parameters:
        compound_name (str): The name of the compound (e.g., "aspirin").

    Returns:
        list: A list of bioassay names or an error message.
    """
    # Step 1: Get the Compound CID
    cid_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/cids/JSON"

    try:
        cid_response = requests.get(cid_url)
        cid_response.raise_for_status()
        cid_data = cid_response.json()

        # Extract first available CID
        cids = cid_data.get("IdentifierList", {}).get("CID", [])
        if not cids:
            return {"error": f"No CID found for compound: {compound_name}"}

        cid = cids[0]  # Use the first CID

        # Step 2: Retrieve Bioassay Summary for the Compound
        bioassay_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/assaysummary/JSON"
        bioassay_response = requests.get(bioassay_url)
        bioassay_response.raise_for_status()
        bioassay_data = bioassay_response.json()

        # Extract assay names
        assays = bioassay_data.get("AssaySummaries", [])
        bioassay_names = [assay.get("Name", f"Assay {assay.get('AID')}") for assay in assays]

        return bioassay_names if bioassay_names else {"error": "No bioassays found for this compound."}

    except requests.exceptions.RequestException as e:
        return {"error": str(e)}


In [3]:
# Import relevant functionality
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.graph import START, MessagesState, StateGraph

# Create the agent
memory = MemorySaver()
model = init_chat_model("gpt-4o-mini", model_provider="openai")
tools = [get_compound_properties, get_bioassay_names]
agent_executor = create_react_agent(model, tools, checkpointer=memory)


In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Can you get me the names of bioassays for this drug")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Question 1
From the above tutorial provided, create your own agent using any api that has memory. You can edit the above cells or add new ones to complete this task. Ask it some relevant questions to your research (or interest). Play around with its functionality and discuss this in a paragraph.

Answer here...

## Part B - Use of Docking Methods for Drug Design

### Sub Part 1 - Docking FDA Approved Compounds to a Target

#### Objective

The object of this part of the assignment is to teach you how to submit CANDOCK jobs. These jobs find the interactions between a library of small of small molecules and a receptor (or target).

#### Procedure

#### Step 1 - Setup CANDOCK

Since CANDOCK's location is non-standard, you will need to tell the computer where to find the executables for CANDOCK. To do this, use the following command:

```
export PATH=/depot/gchopra-class/apps/v0.4.3/modules:$PATH
```

You can add this line to the ~/.bashrc file so that you do not have to run it every time you log into the computer.

In [ ]:
export PATH=/depot/gchopra-class/apps/v0.6.0_chm579/modules:$PATH

You can set up the folder system however you would like for this assignment. I reccomend making one folder for each part. Use the below cell to set up the foldering however you would like or do it on VS Code beforehand.

In [ ]:
# Set up folders here in this cell (or do it on VS Code) ...

#### Step 2 - Obtain a Receptor

For this assignment, we will use the kinase FLT3 as our target. We first need to download this structure and place it in a new directory corresponding to this part of the assignment. The file you downloaded must be renamed as receptor.pdb. If you have completed the previous step, use the following command to do so for the PDB ID 4XUF. Note we will use as an example for future steps.

```
wget https://files.rcsb.org/view/4XUF.pdb 
```

4XUF already has a ligand in it (RCSB will tell you this), you must remove it. There will be a three alphanumeric code for this ligand. In our case it is P30, you can remove it using a text editor, or with this command:

```
cat 4XUF.pdb | grep -v P30 | grep ' A ' > receptor.pdb
```

Note: CANDOCK is able to dock with crystal waters and cofactor (such as metal ions, NAD, etc). Thus, these do NOT need to be removed.

Also note: If you plan on using CANDOCK for your final project, pick a target that may be implicated in a disease pathway that you are interested in studying, or come from a previous assignment for this course. Once this target is selected, search for it on UniProt.org or rcsb.org and find a solved structure for this target (make sure you select a structure with the appropriate domains).

In [ ]:
wget https://files.rcsb.org/view/4XUF.pdb 
cat 4XUF.pdb |grep -v P30 | grep ' A ' > receptor.pdb

#### Step 3 - Determine the Binding Site

In order to dock compounds to a target, one must give a binding site on the target in question. Fortunately, CANDOCK is able to predict binding sites de novo if no binding sites are given by the user. These binding sites are given as text files and are typically named receptor/site.cen (since we called our target receptor) and is placed in a directory named after the target.

If you wish for candock to predict a binding site for you, use the following command, but note that this is not needed as this step will be run automatically.

```
submit_candock_module.sh find_centroids
```

This will create a SLURM job and submit it with sbatch automatically. If this step fails (no receptor/site.cen file is present once the job finishes), you must give a binding site yourself, Contact the instructor if this is the case.

In [ ]:
submit_candock_module.sh find_centroids

**If you are using CANDOCK for your final project, then the next section maybe useful to you. Otherwise, continue to Section 04.**

If you know or are interested in, a binding site for your target create a text file named 'receptor/site.cen' with the following contents. Each line represents a sphere centered around (x,y,z) with radius r. The number in the lefthand column should always be 1 unless you are specifying more than one binding site. This should look like the following example:

1 x1 y1 z1 r1

1 x2 y2 z2 r2

1 x3 y3 z3 r3

....etc....

If you do not know the binding site, do not create any new files. CANDOCK will attempt to determine the binding site automatically and save the result to receptor/site.cen .

#### Step 04 - Prepare the Ligands

Copy the following file to the directory that contains receptor.pdb:

```
cp /depot/gchopra-class/data/lab_7_compounds.mol2 ligands.mol2
prep_fragments.sh
```


In [ ]:
cp /depot/gchopra-class/data/lab_7_compounds.mol2 ligands.mol2
prep_fragments.sh

When the command completes, you should have the following new files: prepared_ligands.pdb, seeds.txt, and seeds.pdb. You can open seeds.pdb in PyMOL to see how CANDOCK fragmented your compounds of interest. If you did this step, skip to step 05.

**If you are using CANDOCK for your final project, the next section may be useful to you. Otherwise, skip to Step 05.**

Note: submitting your own compounds
If you would like to use your own ligands (possibly ones that you find interesting), you can submit a version of your files in the Tripos mol2 format. An online tool exists to convert most formats to mol2 (http://pasilla.health.unm.edu/tomcat/biocomp/convert). Place your ligands as the file ligands.mol2 in the same directory as receptor.pdb. Let's use P30 from 4XUF as an example. Goto rcsb.org and search for the ligand P30. You should find the page: https://www3.rcsb.org/ligand/P30 . Use the button in the top right corner to download the Ideal SDF file. Submit this file to the webserver and download the result. Copy this file as ligands.mol2 in the directly with receptor.pdb in it. Ensure that you do NOT have a prepared_liands.pdb file or a seeds.txt file in this directory! Finally, run the following command (note this assumes you are docking a small number of ligands):
```
prep_fragments.sh
```

#### Step 5 -  Dock the Ligand Fragments

Now that you have a receptor (`receptor.pdb`), a binding site (`receptor/site.cen` will only exist if you ran `find_centroids.sh` or have provided your own), and ligands to dock (`prepared_ligands.pdb`) you can now start docking! We're going to do this in stages so that you understand how the process works.

To dock the fragments, use the following command. Like the step for binding site determination, you do not need to do this explicitly as it will be done automatically in Step 06.

```
submit_candock_module.sh dock_fragments
```

(These are notes for trouble shooting and do not need to be done! Feel free to move onto Step 06)

You will get a new directory called `receptor/top_seeds/` which contains the results of fragment docking performed on the seeds in `seeds.pdb`. You can open the files in this directory using PyMOL to see how CANDOCK placed these fragments in your binding pocket.

Once you have the fragments docked, you can complete the docking procedure by linking the fragments together. Before you begin this step, make sure that the directory top_seeds exists in the directory that you are doing docking in. Also, check the file slurm-###### for any erros.

In [ ]:
submit_candock_module.sh dock_fragments

#### Step 6 - Link the fragments

Technically speaking, this is the only command you need to submit a CANDOCK job as it will run any required previous steps if needed. This is the final step in running CANDOCK and will produce a directory call receptor/docked that will contain the docked conformations of the ligands.

```
submit_candock_module.sh link_fragments
```

Note that for the sake of time, CANDOCK has been configured for this assignment in a way that it will generate fewer conformations than normal. If you are using CANDOCK for your final project, you may want to configure it to generate more. Ask an instructor if this is the case.

In [ ]:
submit_candock_module.sh link_fragments

#### Step 7 - Look at the Results

After the linking step has completed, there will be a directory called `receptor/docked` in the directory that you submitted jobs in. In this directory, a file will be created for all of the compounds that you docked against. These files contain CANDOCK's predicted conformations and corresponding score. Since these files are standard PDB files, they can be opened in PyMOL directly. However, PyMOL has issues with opening these files directly. Thus, it is recommended to follow the following steps:

```
cd receptor/
extract_results.sh
```

You will get two new files in this directory, `scores.csv` which is an Excel file with CANDOCK's predicted scores, and `all.pse` which is a PyMOL sessions. The order of the compounds in this session are in decreasing binding affinity. If a compound is missing from either file, CANDOCK has predicted that this compound is a non-binder.

In [ ]:
cd receptor/
extract_result.sh

You will get two new files in this directory, scores.csv which is an Excel file with CANDOCK's predicted scores, and all.pse which is a PyMOL sessions. The order of the compounds in this session are in decreasing binding affinity. If a compound is missing from either file, CANDOCK has predicted that this compound is a non-binder.

### Sub Part 2 - Design of New Compounds

#### Objective

This assignment will teach you how to use CANDOCK to make changes to existing binders (`lead.mol2`) for a given target using a collection of chemical fragments (`additional.mol2`).

#### Lead optimization

#### Against a single target

A recent paper published by the pharmaceutical company Hoffman La-Roche (citation at the end of section) describes an effort to optimize a DPP-IV inhibitor. The goal was to optimize a lead by replacing an R group with various other fragments. Two files, `lead.mol2` and `additional.mol2` have been prepared with this lead and the additional fragments respectively. Download them and place them in an empty directory. Next, download the protein structure mentioned in the paper (3OC0) and place into the subdirectory targets/ (remember to clean it up first!). Then create the following script and run it.

```
cp /depot/gchopra-class/data/additional.mol2 .
cp /depot/gchopra-class/data/lead.mol2 .

mkdir targets
cp /depot/gchopra-class/data/target.pdb targets/

export CANDOCK_ligand=lead.mol2
export CANDOCK_fragment_mol=additional.mol2

prep_fragments.sh
export CANDOCK_target_dir=targets
export CANDOCK_lipinski_nhs=5
export CANDOCK_lipinski_ohs=5


submit_candock_module.sh design_ligands
```

You will get the files `designed_1.mol2` and `designed2.mol2`. Do they match the compounds suggested by the authors?

A Real-World Perspective on Molecular Design Kuhn, B. et al. Journal of Medicinal Chemistry 2016 59 (9), 4087-4102 DOI: 10.1021/acs.jmedchem.5b01875

In [ ]:
# Add bash code for organization here ...

In [ ]:
cp /depot/gchopra-class/data/additional.mol2 .
cp /depot/gchopra-class/data/lead.mol2 .

mkdir targets
cp /depot/gchopra-class/data/target.pdb targets/

export CANDOCK_ligand=lead.mol2
export CANDOCK_fragment_mol=additional.mol2

prep_fragments.sh
export CANDOCK_target_dir=targets
export CANDOCK_lipinski_nhs=5
export CANDOCK_lipinski_ohs=5


submit_candock_module.sh design_ligands

#### Against multiple targets

The procedure for profile based design is similar to the one for single targets, the only difference is that one places more targets in the targets/ directory. To give CANDOCK targets that you wish to decrease binding affinity to, you create an additional folder called atargets. **Create a new directory from scratch (IE separate from any previous jobs) and run the following commands**:

```
cp /depot/gchopra-class/data/additional.mol2 .
cp /depot/gchopra-class/data/lead.mol2 .

mkdir targets
cp /depot/gchopra-class/data/target.pdb targets/

mkdir atargets
cp /depot/gchopra-class/data/antitarget.pdb atargets/

export CANDOCK_ligand=lead.mol2
export CANDOCK_fragment_mol=additional.mol2

prep_fragments.sh

export CANDOCK_target_dir=targets
export CANDOCK_antitarget_dir=atargets
export CANDOCK_lipinski_nhs=5
export CANDOCK_lipinski_ohs=5
export CANDOCK_seeds_to_avoid=2

submit_candock_module.sh design_ligands
```


In [ ]:
# Add bash code for organization here ...

In [ ]:
cp /depot/gchopra-class/data/additional.mol2 .
cp /depot/gchopra-class/data/lead.mol2 .

mkdir targets
cp /depot/gchopra-class/data/target.pdb targets/

mkdir atargets
cp /depot/gchopra-class/data/antitarget.pdb atargets/

export CANDOCK_ligand=lead.mol2
export CANDOCK_fragment_mol=additional.mol2

prep_fragments.sh

export CANDOCK_target_dir=targets
export CANDOCK_antitarget_dir=atargets
export CANDOCK_lipinski_nhs=5
export CANDOCK_lipinski_ohs=5
export CANDOCK_seeds_to_avoid=2

submit_candock_module.sh design_ligands

#### Design of new scaffolds

**Note: you can ignore this section of assignment. It is presented to you only in hope that it may help you for your final project. It is not required for this lab.**

CANDOCK has the ability to design new compounds from the seeds database generated in Step 05. Create a new directory (separate from the one you did docking in) and copy the files seeds.pdb and seeds.txt into this new directory. Create a subdirectory called targets in this location and and copy the receptor directory from your previous run into this subdirectory along with `receptor.pdb`. The directory structure should look like the following:

```
$ ls

seeds.txt seeds.pdb targets/

$ ls targets/

receptor.pdb receptor/

$ ls targets/receptor/

top_seeds/ site.cen gridpdb_hcp.pdb

export CANDOCK_target_dir=targets
submit_candock_module.sh design_ligands
```
Congrats! Now you're creating new designs! When the job completes, you will get files named like `designed#.pdb`. These files contain the designs generated by the program. The docked version of these compounds is present in `targets/receptor/docked/` . You can use the extract script to get the scores of these compounds.

[CANDOCK README File](https://web.ics.purdue.edu/~gchopra/class/public/assignments/lab4c/README.html)

[CANDOCK Paper](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.9b00686#)

### Questions

#### Sub Part 1 

1. For your writeup, you should include images of the docked poses and docking scores of the compound. 
    - Which drugs bind best to your protein of choice? 
    - Did you expect this? Why or why not? 
    - Note that the three letter codes ( P30, KIM, etc ) are all kinase inhibitors!
2. Include images of the interactions between some of the high scoring molecules and the low scoring ones. 
    - What interactions do you believe helped make the higher scoring compounds bind better than the low scoring ones?
3. Open all.pse along with the original 4XUF file. 
    - How well did CANDOCK predict the binding pose of P30? 
    - Using the arrows in PyMOL's window, checkout the multiple poses generated by CANDOCK. 
    - Report CANDOCK scores of the well predicted pose and the top pose of the P30 molecule
4. Part of CANDOCK's linking procedure includes an energy minimization procedure. 
    - Did your target's conformation change while docking? If so, by much?

#### Sub Part 2

1. Examine the designs created by CANDOCK. 
    - Do they match what is listed in the paper?
2. How does the addition of the antitarget change the designs created?
3. Include images of some of the designed ligands and discuss their interactions at the binding site

### Part C - Combined molecular Graph Neural Networks and Structural Docking to Select Potent PD-1/PD-L1 Small Molecule Inhibitors

#### Introduction

The Programmable Cell Death Protein 1/Programmable Death-Ligand 1 (PD-1/PD-L1) interaction is an immune checkpoint utilized by cancer cells to enhance immune suppression. There exists a huge need to develop small molecule drugs that are fast acting, cheap, and readily bioavailable compared to antibodies. Unfortunately, synthesizing and validating large libraries of small-molecule to inhibit PD-1/PD-L1 interaction in a blind manner is a both time-consuming and expensive. Here we use a machine learning method call EGNN which is based on graph neural networks (GNNs) and docking scores. We call them local and global features respectively. GNNs are models which use message passing between nodes and edges of graphs. These graphs can be anything. In our case nodes are atoms and edges are bonds of molecules. GNNs retain a state with information of its neighbours with an arbitrary depth. We call it as the sub graph radius. On the other hand, docking scores can be used as global features which captures interactions of the molecules with the target. Therefore, EGNN model take both local and global features to select potent small molecule inhibitors.

[Graph Neural Networks - A Review of Methods and Applications](https://arxiv.org/pdf/1812.08434.pdf)

#### Objective

This assignment will teach you how to use machine learning and molecular modeling, specifically Graph Neural Networks in combination with molecular docking, to select potent small molecule PD-1/PD-L1 inhibitors.

#### Procedure

#### Step 1a -  Downloading required files and scripts

All the files and scripts required for the assignment can be downloaded from here. Make a sub directory in your working directory for this assignment. Make folders fullmodel/, input/, test/, train/ in this sub directory and copy all the scripts into the subdirectory. Now you should have four folders and downloaded scripts in your working directory. Now place the data.txt and test.txt files in the input/ directory. The data.txt file has SMILES, 96 CANDOCK docking scores against PD-L1 homodimer and their active or inactive status. The test.txt file also have SMILES and docking scores. Here, information in the data.txt file will be used to train the machine learning model and the trained machine learning model (EGNN) will be used to predict the activity of a molecular library in test.txt.

Link to download EGNN Model
http://chopra-modules.science.purdue.edu/class/chm579/spring2020/public/assignments/lab6_2020/lab6b/egnn-master.zip 

#### Step 1b - Creating a Python environment and install required packages

**Don't run any commands for this in the Jupyter Notebook, you should run them in the VS Code terminal as it the bash kernel will not interface well with the Conda module on Scholar** 

Run commands below to create the python environment and install packages

```
module load anaconda
rcac-conda-env create -n egnn -j
source activate egnn
conda install pytorch -c pytorch
conda install -c rdkit rdkit
conda install -c anaconda scikit-learn
conda install pandas==1.5.3
```

#### Step 2 - Preprocessing data

This step will preprocess the given data. It will open both `data.txt` and `test.txt` files and save the data separately as needed to run the training step. Here we can change the sub-graph radius with in the `preprocess_data.sh` script. It has been set to 2 by default. Following command can be used to do this.

```
bash preprocess_data.sh
```

If running the script is successful, you will see some prepared files in the train/ folder.

#### Step 3 - Run training

Open and see the parameters in the `run_training.sh` script. You should keep the same sub-graph radius which you used for pre-processing data. Keep the other parameters as defaults. You can use following command to start training the model.

```
bash run_training.sh
```

After training is finished you will see a model file and a text file in the fullmodel/ directory. The text file will have the epoch, time, training loss, validation loss, area under the curve (AUC), precision, recall and F1 score.

#### Step 4 - Train full with bootstrapping and take predictions for the test set

Use the following command to start training with bootstrapping. This will train the model hundred times with hundred different random seeds and record all the data in the bootstrapping_results directory. You should keep same parameters which you used in step 03.

```
sbatch train_full.sh
```

After training with bootstrapping is done, create a subdirectory called `bootstrapping_results/` in the working directory. Then use the following command to take predictions. Again keep same parameters.

```
bash run_test.sh
```

A csv file with all the predictions for the test set will be generated in the working directory. The take_counts.py script can be used to calculate the number of times a molecule to be predicted as active over number of bootstrapped models. Then this number can be combined with main results using the script `combining_smiles_bootstrapping_average_countsover0.5.py`. Sripts can be used as follows.

### Questions

1. Change the dim (molecular vector dimension) parameter in the run_training.sh script and run it for 6 different numbers. (eg: You can try dim = 6, 7, 8, 9, 10, 15). Then plot average F1 score against epoch for these dim parameter values in the same plot.

2. Generate similar plots for radius parameter (radius = 2 and 3 ) and hidden_layer parameter. Determine most suitable parameters to be used in the training of EGNN for this data based on the graphs which you have plotted.

3. Calculate and report the average training F1 score, average validation F1 score, average training AUC and average validation AUC for the EGNN model over five folds.

4. Give structures of the top five predictions you got as final results with their respective average softmax scores with the standard deviation.

5. What is the effect of bootstrapping in EGNN?